In [ ]:
import pandas as pd
import glob
import json
import dotted # https://pypi.org/project/dotted-notation/
import re
import matplotlib.pyplot as plt
import itertools
import numpy as np

from pathlib import Path
import seaborn as sns
import lib.datasciencetoolbelt as dstools
from lib.resultstorage import ResultStorage

In [ ]:
dstools.setup({
    "seaborn_context": "talk",
    "savefig": {
        "enable": True,
        "dir": Path("./postprocess_results"),
    }
})
result_storage = ResultStorage(Path("./results"))

#%matplotlib qt
%matplotlib inline


In [ ]:
result_storage_prefix = "fs_comparison_v2"

def to_row(d):
    
    if dotted.get(d, 'result.exception') is True:
        #print(f"skipping benchmark that threw exception: {d}")
        return None
    
    if dotted.get(d, 'result.dummy') is not None:
        return None # skip dummy
    
  
    
    blockdev_stack = dotted.get(d, 'storage_stack.blockdev_stack.identity')
    if blockdev_stack is None:
        blockdev_stack = 'native'
    
    storage_stack = dotted.get(d, 'storage_stack.fstyp')
    if storage_stack is not None:
        storage_stack += "__on__" + blockdev_stack
        storage_stack += "__dax_" + str(dotted.get(d, 'storage_stack.mount_dax'))
    else:
        storage_stack = dotted.get(d, 'storage_stack.identity')
    assert storage_stack is not None
    
    benchmark_identity = dotted.get(d, 'result.identity')
    
    # fixup mariadb-sysbench-oltp_insert
    if benchmark_identity == "mariadb-sysbench-oltp_insert":
        m = d['result']['result']['metrics']
        m['SQL_statistics'] = m['SQL statistics']
        del m
    
    # determine variable_dotted_str and metric_dotted_strs
    try:
        variable_dotted_str, metric_dotted_strs = {
            "filebench-varmail": ('result.config.vars.nthreads', ['result.result.metrics.summary_ops_per_sec']),
            "filebench-oltp": ('result.config.vars.ndbwriters', ['result.result.metrics.summary_ops_per_sec']),
            "redis-SET": ('result.config.clients', ['result.result.main.metrics.rps']),
            "rocksdb-fillsync": ('result.config.threads', ['result.result.metrics.fillsync.ops_per_sec']),
            "sqlite-bench": (None, ['result.result.metrics.fillrandsync.micros_per_op']),
            "mariadb-sysbench-oltp_insert": ('result.config.threads', ['result.result.metrics.SQL_statistics.transactions_per_sec']),
            "fio-4k-sync-rand-write": ('result.fio_config.numjobs', ['result.fio_jsonplus.jobs[0].write.iops']),
        }[benchmark_identity]
    except KeyError:
        raise Exception(f"unknown_benchmark {benchmark_identity}:\n{d!r}")
    
    # determine variable_value
    if variable_dotted_str:
        variable_value = dotted.get(d, variable_dotted_str)
        if variable_value is None:
            print(benchmark_identity)
            raise Exception(str(d))
    else:
        variable_value = None
        
    # determine metric_value
    if not metric_dotted_strs:
        raise Exception(str(d))
    metrics = {}
    for mds in metric_dotted_strs:
        mv = dotted.get(d, mds)
        if not mv:
            raise Exception(f"mds: {mds}\n{d!r}")
        if len(metrics) == 0:
            metrics['primary_metric'] = mds
            metrics['primary_metric_value'] = mv
        metrics[mds] = mv 
    
    return {
        "storage_stack": storage_stack,
        "blockdev_stack": blockdev_stack,
        "benchmark": benchmark_identity,
        "variable": variable_dotted_str,
        "variable_value": variable_value,
        **metrics,
    }

rows = [to_row(j) for j in result_storage.iter_results(result_storage_prefix)]
rows = list(filter(lambda d: d is not None, rows))
rows = list(itertools.chain(rows))
df = pd.DataFrame(rows)
df


In [ ]:
import itertools

def filter_by_index_value(df, level, filter):
    """Return a new df that only contains rows whose MultiIndex column `level`'s value passes `filter`"""
    return df[df.index.get_level_values(level).map(filter)]

def remove_index_dimension(df, level, value):
    """Reduce dimensionality of a dataframe by filtering by and subsequently dropping one of its index levels.
    
    df is assumed to be a multi-indexed pd.DataFrame.
    First, filter the data frame so that we only keep rows whose index tuple has value `value` at level `level`.
    Now the resulting data frame only has a single value at the level.
    Thus remove that level from the index.
    Voila: dimensionality reduced.
    """
    df = df[df.index.get_level_values(level) == value]
    assert set(df.index.get_level_values(level)) == {value}
    df.index = df.index.droplevel(level)
    return df

def _test_remove_index_dimension():
    data = [{"favnum": n, "favletter": l, "id": id} for id, (n, l) in enumerate(itertools.product([23,42],["a", "b"]))]
    d = pd.DataFrame(data).set_index(["favnum", "favletter"])
    display(d)
    display(remove_index_dimension(d, "favnum", 23))
    display(remove_index_dimension(d, "favletter", "b"))
    
_test_remove_index_dimension()

In [ ]:
def level_values_sorted_unique(df, level):
    """Returns the sorted unique values of a DataFrame's multi-index at level `level`"""
    return sorted(list(set(df.index.get_level_values(level))))

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
        
class FactorizedDataFrameItem(AttrDict):
    @property
    def title(self):
        if self.fdf.row and self.fdf.col:
            return f"{self.fdf.row}={self.rv}|{self.fdf.col}={self.cv}"
        elif self.fdf.row:
            return f"{self.fdf.row}={self.rv}"
        elif self.fdf.col:
            return f"{self.fdf.col}={self.cv}"
        else:
            return ""
            
        
class FactorizedDataFrame:
    def __init__(self, data, row, col):
        self.data = data
        self.col = col
        self.row = row

        self.col_values = [None] if not self.col else level_values_sorted_unique(self.data, self.col)
        self.row_values = [None] if not self.row else level_values_sorted_unique(self.data, self.row)
        
    def iter_factorized(self):
        for ci, c in enumerate(self.col_values):
            for ri, r in enumerate(self.row_values):
                d = self.data.copy()
                if c:
                    d = remove_index_dimension(d, self.col, c)
                if r:
                    d = remove_index_dimension(d, self.row, r)
                # display(d)
            
                context = FactorizedDataFrameItem({
                    "fdf": self,
                    "d": d,
                    "ri": ri,
                    "rv": r,
                    "ci": ci,
                    "cv": c,
                })
                yield context
                

def factorplot(data=None, row=None, col=None, plot=None, subplots_kw={}):
    """Factorizez MultiIndex'ed DataFrame `data`, then invokes `plot` for each FactorizedDataFrameItem"""
    
    fdf = FactorizedDataFrame(data, row, col)
    
    subplots_kw = {
        "gridspec_kw": {'hspace': 1},
        **subplots_kw,
        "squeeze": False, # axes should always be two-dimensional
    }

    fig, axes = plt.subplots(len(fdf.row_values), len(fdf.col_values), **subplots_kw)

    for f in fdf.iter_factorized():
        ax = axes[f.ri, f.ci]
        ax.set_title(f.title)
        legend = f.ri == len(fdf.row_values)-1 and f.ci == len(fdf.col_values)-1
        plot(f, ax, legend)
        if legend:
            plt.legend(loc='lower left', bbox_to_anchor=(1,0.5))

In [ ]:
set(df['benchmark'])

In [ ]:
set(df['storage_stack'])

In [ ]:
set(df['blockdev_stack'])

In [ ]:
#data = df.copy().query('variable_value in [1, @nan] and benchmark in ["redis-SET", "filebench-varmail"]')
nan = np.nan
data = df.copy().query('variable_value in [1, @nan]')
# data = df.copy().query('variable_value in [1, @nan] and benchmark in ["redis-SET", "filebench-varmail", "rocksdb-fillsync"]')
# display(data)
data = data.set_index(['benchmark', 'storage_stack', 'variable_value'], verify_integrity=True).sort_index()
data

In [ ]:
def compare_benchmarks(storage_stacks, baseline=None):
    
    # uncomment this if absolute values are of interest
    #     if baseline:
    #         compare_benchmarks(storage_stacks, baseline=None)
    
    data = df.copy()
    
    data = data.query('benchmark not in ["sqlite-bench"]').copy()

#     tmpl = data.loc[data.benchmark == 'sqlite-bench'].copy()
#     for variable_value in range(0, 17):
#         tmp = tmpl.copy()
#         tmp.variable_value = variable_value
#         tmp.variable = 'fake'
#         if variable_value != 1:
#             tmp.primary_metric_value = 0
#         data = data.append(tmp)

    # data.loc[data.benchmark == 'sqlite-bench', 'variable_value'] = 1.0
    # data.loc[data.benchmark == 'sqlite-bench', 'variable'] = 'constant'

#     data = data.loc[data.blockdev_stack.map(lambda v: v in blockdev_stacks)]

    data = data[data.storage_stack.map(lambda v: v in storage_stacks)]
    # data = data[data.storage_stack.map(lambda v: "byp_1" not in v)]

    data = data.set_index(['benchmark', 'storage_stack', 'variable_value'], verify_integrity=True)
    # data.loc[('sqlite-bench'), 'variable'] = 'constant'
    # display(data.loc[('sqlite-bench'), ].index.columns['variable_value'])

    # data = data.query('variable_value in [1, 2, 3, 4, 5, 6, 7, 8]')

    #data = df.copy().query('variable_value in [1, @nan] and benchmark in ["redis-SET", "filebench-varmail"]')
    # data = df.copy().query('variable_value in [1, @nan]')
    # data = df.copy().query('variable_value in [1, @nan] and benchmark in ["redis-SET", "filebench-varmail", "rocksdb-fillsync"]')
    # display(data)
    data
    
    def plot(f, ax, legend):
    #     f.d['primary_metric_value'].unstack('benchmark').plot.bar(ax=ax, ylim=(0, 1.1*f.fdf.data.max().max()))
    #     return
    #     display(f.d)

        d = f.d.copy()

#         if baseline:
#             bl = remove_index_dimension(d.copy(), 'storage_stack', baseline)
# #         baseline = remove_index_dimension(f.d.copy(), 'storage_stack', 'xfs__on__devpmem__dax_True')
# #             print("baseline", bl)
# #             print(d)
#             d = d.div(bl, axis=0)
        
        bars = "storage_stack"
        d = d.unstack(bars)
        d = d.sort_values(bars, axis=1)
        d = d.sort_index()
#         display(d.T)
    
        d['primary_metric_value'].plot.bar(ax=ax, legend=legend, ylim=(0, 1.1*f.fdf.data.max().max()))
        
        renames = {
            "filebench-oltp":"fb-oltp",
            "filebench-varmail": "fb-varmail",
            'fio-4k-sync-rand-write': 'fio-4k-randrw',
            'mariadb-sysbench-oltp_insert': 'MariaDB/sysbench',
            'rocksdb-fillsync': 'RocksDB-fillsync',
            'redis-SET': 'Redis-SET',
        }
        renamedlabels = []
        for text in ax.get_xticklabels():
            name = text.get_text()
            text.set_text(renames.get(name, name))
            renamedlabels += [text]
        ax.set_xticklabels(renamedlabels, rotation='horizontal')
        
        if not legend:
            ax.set_xticklabels([])
            ax.set_xlabel("")
            
    tmp = pd.DataFrame(data['primary_metric_value'])
    
    if baseline:
        bl = remove_index_dimension(tmp.copy(), 'storage_stack', baseline)
        tmp = tmp.div(bl, axis=0)
        
    tmp = tmp.query('variable_value in [1, 4, 8]')
    
    
#     tmp = tmp.div(tmp.index.get_level_values('variable_value'), axis=0)
    #     display()

    factorplot(tmp, 'variable_value', None, plot=plot, subplots_kw={
        "figsize": (15, 5 + 2*1),
#         "figsize": (10, (0.6666) * (5 + 3*1)),
        "gridspec_kw": {
            "hspace": 0.2,
        },
    })

# First take a hard look on `ncommitters`, `zvol_request_sync` and `bypass` to determine which we include in the subsequent graphs. This graph is not included in the publication.

* nc2 vs nc3 (neighboring bars): hardly any difference, use nc3
* rs0 vs rs1
  * generally not much of a difference
  * ext4 filebench-varmail performs significantly better with rs0 vs rs1
* byp0 vs byp1: effect noticable at higher thread counts


Note that the effect of ITXG bypass in the Block-Device-Proivder-Role is noticable but not significant (maybe this changes if we add a `fio` benchmark?)

In [ ]:
tmp = {
#  'ext4__on__devpmem__dax_False',
#  'ext4__on__devpmem__dax_True',
#  'ext4__on__dm-writecache__dax_False',
#  'ext4__on__zvol-lwb-rs_0__dax_False',
#  'ext4__on__zvol-lwb-rs_1__dax_False',
 'ext4__on__zvol-pmem-rs_0-byp_0-nc_2__dax_False',
 'ext4__on__zvol-pmem-rs_0-byp_0-nc_3__dax_False',
 'ext4__on__zvol-pmem-rs_0-byp_1-nc_2__dax_False',
 'ext4__on__zvol-pmem-rs_0-byp_1-nc_3__dax_False',
 'ext4__on__zvol-pmem-rs_1-byp_0-nc_2__dax_False',
 'ext4__on__zvol-pmem-rs_1-byp_0-nc_3__dax_False',
 'ext4__on__zvol-pmem-rs_1-byp_1-nc_2__dax_False',
 'ext4__on__zvol-pmem-rs_1-byp_1-nc_3__dax_False',
#  'xfs__on__devpmem__dax_False',
#  'xfs__on__devpmem__dax_True',
#  'xfs__on__dm-writecache__dax_False',
#  'xfs__on__zvol-lwb-rs_0__dax_False',
#  'xfs__on__zvol-lwb-rs_1__dax_False',
 'xfs__on__zvol-pmem-rs_0-byp_0-nc_2__dax_False',
 'xfs__on__zvol-pmem-rs_0-byp_0-nc_3__dax_False',
 'xfs__on__zvol-pmem-rs_0-byp_1-nc_2__dax_False',
 'xfs__on__zvol-pmem-rs_0-byp_1-nc_3__dax_False',
 'xfs__on__zvol-pmem-rs_1-byp_0-nc_2__dax_False',
 'xfs__on__zvol-pmem-rs_1-byp_0-nc_3__dax_False',
 'xfs__on__zvol-pmem-rs_1-byp_1-nc_2__dax_False',
 'xfs__on__zvol-pmem-rs_1-byp_1-nc_3__dax_False',
#  'zfs-lwb-rs_0',
#  'zfs-lwb-rs_1',
#  'zfs-pmem-rs_0-byp_0-nc_2',
#  'zfs-pmem-rs_0-byp_0-nc_3',
#  'zfs-pmem-rs_0-byp_1-nc_2',
#  'zfs-pmem-rs_0-byp_1-nc_3',
#  'zfs-pmem-rs_1-byp_0-nc_2',
#  'zfs-pmem-rs_1-byp_0-nc_3',
#  'zfs-pmem-rs_1-byp_1-nc_2',
#  'zfs-pmem-rs_1-byp_1-nc_3'
}
compare_benchmarks(list(tmp), baseline='ext4__on__zvol-pmem-rs_0-byp_0-nc_2__dax_False')

# ZFS+ZIL-PMEM vs. ZFS+ZIL-LWB  vs. XFSonPMEM vs. Ext4onPMEM

* ZIL-PMEM has very high speedup over ZIL-LWB
* In the very sync-IO heavy workloads (redis-SET, rocksdb-fillsync), ZIL-PMEM outperforms XFS, which is the Linux FS that performs best on PMEM

In [ ]:
tmp = {
 'ext4__on__devpmem__dax_False',
 'ext4__on__devpmem__dax_True',
#  'ext4__on__dm-writecache__dax_False',
#  'ext4__on__zvol-lwb-rs_0__dax_False',
#  'ext4__on__zvol-lwb-rs_1__dax_False',
    
#  'ext4__on__zvol-pmem-rs_0-byp_0-nc_2__dax_False',
#  'ext4__on__zvol-pmem-rs_0-byp_0-nc_3__dax_False',
#  'ext4__on__zvol-pmem-rs_0-byp_1-nc_2__dax_False',
#  'ext4__on__zvol-pmem-rs_0-byp_1-nc_3__dax_False',
#  'ext4__on__zvol-pmem-rs_1-byp_0-nc_2__dax_False',
#  'ext4__on__zvol-pmem-rs_1-byp_0-nc_3__dax_False',
#  'ext4__on__zvol-pmem-rs_1-byp_1-nc_2__dax_False',
#  'ext4__on__zvol-pmem-rs_1-byp_1-nc_3__dax_False',
    
 'xfs__on__devpmem__dax_False',
 'xfs__on__devpmem__dax_True',
#  'xfs__on__dm-writecache__dax_False',
#  'xfs__on__zvol-lwb-rs_0__dax_False',
#  'xfs__on__zvol-lwb-rs_1__dax_False',
    
#  'xfs__on__zvol-pmem-rs_0-byp_0-nc_2__dax_False',
#  'xfs__on__zvol-pmem-rs_0-byp_0-nc_3__dax_False',
#  'xfs__on__zvol-pmem-rs_0-byp_1-nc_2__dax_False',
#  'xfs__on__zvol-pmem-rs_0-byp_1-nc_3__dax_False',
#  'xfs__on__zvol-pmem-rs_1-byp_0-nc_2__dax_False',
#  'xfs__on__zvol-pmem-rs_1-byp_0-nc_3__dax_False',
#  'xfs__on__zvol-pmem-rs_1-byp_1-nc_2__dax_False',
#  'xfs__on__zvol-pmem-rs_1-byp_1-nc_3__dax_False',
    
 'zfs-lwb-rs_0',
#  'zfs-lwb-rs_1',
#  'zfs-pmem-rs_0-byp_0-nc_2',
 'zfs-pmem-rs_0-byp_0-nc_3',
#  'zfs-pmem-rs_0-byp_1-nc_2',
#  'zfs-pmem-rs_0-byp_1-nc_3',
#  'zfs-pmem-rs_1-byp_0-nc_2',
#  'zfs-pmem-rs_1-byp_0-nc_3',
#  'zfs-pmem-rs_1-byp_1-nc_2',
#  'zfs-pmem-rs_1-byp_1-nc_3'
}
compare_benchmarks(list(tmp), baseline='zfs-pmem-rs_0-byp_0-nc_3')

# Virtual Block Devices (writecache vs zvol-lwb vs zvol-pmem)

* ZIL-PMEM delivers significant speedup over ZIL-LWB
* ZIL-PMEM is competitive to dm-writecache, slightly lower throughput sometimes
* Effect of Bypass
  * shows to some extent for xfs
  * For ext4 in `filebench-oltp` at 8 threads, there is a huge decline

In [ ]:
tmp = {
#  'ext4__on__devpmem__dax_False',
#  'ext4__on__devpmem__dax_True',
 'ext4__on__dm-writecache__dax_False',
 'ext4__on__zvol-lwb-rs_0__dax_False',
 'ext4__on__zvol-lwb-rs_1__dax_False',
    
#  'ext4__on__zvol-pmem-rs_0-byp_0-nc_2__dax_False',
 'ext4__on__zvol-pmem-rs_0-byp_0-nc_3__dax_False',
#  'ext4__on__zvol-pmem-rs_0-byp_1-nc_2__dax_False',
 'ext4__on__zvol-pmem-rs_0-byp_1-nc_3__dax_False',
#  'ext4__on__zvol-pmem-rs_1-byp_0-nc_2__dax_False',
 'ext4__on__zvol-pmem-rs_1-byp_0-nc_3__dax_False',
#  'ext4__on__zvol-pmem-rs_1-byp_1-nc_2__dax_False',
 'ext4__on__zvol-pmem-rs_1-byp_1-nc_3__dax_False',
    
#  'xfs__on__devpmem__dax_False',
#  'xfs__on__devpmem__dax_True',
 'xfs__on__dm-writecache__dax_False',
 'xfs__on__zvol-lwb-rs_0__dax_False',
 'xfs__on__zvol-lwb-rs_1__dax_False',
    
#  'xfs__on__zvol-pmem-rs_0-byp_0-nc_2__dax_False',
 'xfs__on__zvol-pmem-rs_0-byp_0-nc_3__dax_False',
#  'xfs__on__zvol-pmem-rs_0-byp_1-nc_2__dax_False',
 'xfs__on__zvol-pmem-rs_0-byp_1-nc_3__dax_False',
#  'xfs__on__zvol-pmem-rs_1-byp_0-nc_2__dax_False',
 'xfs__on__zvol-pmem-rs_1-byp_0-nc_3__dax_False',
#  'xfs__on__zvol-pmem-rs_1-byp_1-nc_2__dax_False',
 'xfs__on__zvol-pmem-rs_1-byp_1-nc_3__dax_False',
    
#  'zfs-lwb-rs_0',
#  'zfs-lwb-rs_1',
#  'zfs-pmem-rs_0-byp_0-nc_2',
#  'zfs-pmem-rs_0-byp_0-nc_3',
#  'zfs-pmem-rs_0-byp_1-nc_2',
#  'zfs-pmem-rs_0-byp_1-nc_3',
#  'zfs-pmem-rs_1-byp_0-nc_2',
#  'zfs-pmem-rs_1-byp_0-nc_3',
#  'zfs-pmem-rs_1-byp_1-nc_2',
#  'zfs-pmem-rs_1-byp_1-nc_3'
}
compare_benchmarks(tmp)
compare_benchmarks(list(filter(lambda s: "ext4" in s, tmp)), baseline='ext4__on__zvol-pmem-rs_0-byp_0-nc_3__dax_False')
compare_benchmarks(list(filter(lambda s: "xfs" in s, tmp)), baseline='xfs__on__zvol-pmem-rs_0-byp_0-nc_3__dax_False')

# DM-Writecache vs. ZFS (ext+writecache, xfs+writecache, zfs+lwb, zfs+pmem)

* only need to look at zfs rs0 byp0 since neither are relevant for ZPL (only ZVOL)

**ZIL-PMEM is better than dm-writecache**

In [ ]:
tmp = {
#  'ext4__on__devpmem__dax_False',
#  'ext4__on__devpmem__dax_True',
 'ext4__on__dm-writecache__dax_False',
#  'ext4__on__zvol-lwb-rs_0__dax_False',
#  'ext4__on__zvol-lwb-rs_1__dax_False',
    
#  'ext4__on__zvol-pmem-rs_0-byp_0-nc_2__dax_False',
#  'ext4__on__zvol-pmem-rs_0-byp_0-nc_3__dax_False',
#  'ext4__on__zvol-pmem-rs_0-byp_1-nc_2__dax_False',
#  'ext4__on__zvol-pmem-rs_0-byp_1-nc_3__dax_False',
#  'ext4__on__zvol-pmem-rs_1-byp_0-nc_2__dax_False',
#  'ext4__on__zvol-pmem-rs_1-byp_0-nc_3__dax_False',
#  'ext4__on__zvol-pmem-rs_1-byp_1-nc_2__dax_False',
#  'ext4__on__zvol-pmem-rs_1-byp_1-nc_3__dax_False',
    
#  'xfs__on__devpmem__dax_False',
#  'xfs__on__devpmem__dax_True',
 'xfs__on__dm-writecache__dax_False',
#  'xfs__on__zvol-lwb-rs_0__dax_False',
#  'xfs__on__zvol-lwb-rs_1__dax_False',
    
#  'xfs__on__zvol-pmem-rs_0-byp_0-nc_2__dax_False',
#  'xfs__on__zvol-pmem-rs_0-byp_0-nc_3__dax_False',
#  'xfs__on__zvol-pmem-rs_0-byp_1-nc_2__dax_False',
#  'xfs__on__zvol-pmem-rs_0-byp_1-nc_3__dax_False',
#  'xfs__on__zvol-pmem-rs_1-byp_0-nc_2__dax_False',
#  'xfs__on__zvol-pmem-rs_1-byp_0-nc_3__dax_False',
#  'xfs__on__zvol-pmem-rs_1-byp_1-nc_2__dax_False',
#  'xfs__on__zvol-pmem-rs_1-byp_1-nc_3__dax_False',
    
 'zfs-lwb-rs_0',
#  'zfs-lwb-rs_1',
#  'zfs-pmem-rs_0-byp_0-nc_2',
 'zfs-pmem-rs_0-byp_0-nc_3',
#  'zfs-pmem-rs_0-byp_1-nc_2',
#  'zfs-pmem-rs_0-byp_1-nc_3',
#  'zfs-pmem-rs_1-byp_0-nc_2',
#  'zfs-pmem-rs_1-byp_0-nc_3',
#  'zfs-pmem-rs_1-byp_1-nc_2',
#  'zfs-pmem-rs_1-byp_1-nc_3'
}
compare_benchmarks(list(tmp), baseline='zfs-pmem-rs_0-byp_0-nc_3')

# If we are only allowed one Plot

In [ ]:
tmp = {
 'ext4__on__devpmem__dax_False',
 'ext4__on__devpmem__dax_True',
 'ext4__on__dm-writecache__dax_False',
#  'ext4__on__zvol-lwb-rs_0__dax_False',
#  'ext4__on__zvol-lwb-rs_1__dax_False',
    
#  'ext4__on__zvol-pmem-rs_0-byp_0-nc_2__dax_False',
#  'ext4__on__zvol-pmem-rs_0-byp_0-nc_3__dax_False',
#  'ext4__on__zvol-pmem-rs_0-byp_1-nc_2__dax_False',
#  'ext4__on__zvol-pmem-rs_0-byp_1-nc_3__dax_False',
#  'ext4__on__zvol-pmem-rs_1-byp_0-nc_2__dax_False',
#  'ext4__on__zvol-pmem-rs_1-byp_0-nc_3__dax_False',
#  'ext4__on__zvol-pmem-rs_1-byp_1-nc_2__dax_False',
#  'ext4__on__zvol-pmem-rs_1-byp_1-nc_3__dax_False',
    
 'xfs__on__devpmem__dax_False',
 'xfs__on__devpmem__dax_True',
 'xfs__on__dm-writecache__dax_False',
#  'xfs__on__zvol-lwb-rs_0__dax_False',
#  'xfs__on__zvol-lwb-rs_1__dax_False',
    
#  'xfs__on__zvol-pmem-rs_0-byp_0-nc_2__dax_False',
#  'xfs__on__zvol-pmem-rs_0-byp_0-nc_3__dax_False',
#  'xfs__on__zvol-pmem-rs_0-byp_1-nc_2__dax_False',
#  'xfs__on__zvol-pmem-rs_0-byp_1-nc_3__dax_False',
#  'xfs__on__zvol-pmem-rs_1-byp_0-nc_2__dax_False',
#  'xfs__on__zvol-pmem-rs_1-byp_0-nc_3__dax_False',
#  'xfs__on__zvol-pmem-rs_1-byp_1-nc_2__dax_False',
#  'xfs__on__zvol-pmem-rs_1-byp_1-nc_3__dax_False',
    
 'zfs-lwb-rs_0',
#  'zfs-lwb-rs_1',
#  'zfs-pmem-rs_0-byp_0-nc_2',
 'zfs-pmem-rs_0-byp_0-nc_3',
#  'zfs-pmem-rs_0-byp_1-nc_2',
#  'zfs-pmem-rs_0-byp_1-nc_3',
#  'zfs-pmem-rs_1-byp_0-nc_2',
#  'zfs-pmem-rs_1-byp_0-nc_3',
#  'zfs-pmem-rs_1-byp_1-nc_2',
#  'zfs-pmem-rs_1-byp_1-nc_3'
}
compare_benchmarks(list(tmp), baseline='zfs-pmem-rs_0-byp_0-nc_3')